## **_Using News Data to Predict Movements in the Financial Movements_**

We'll be using two apporaches here:

* Continuous Bag of Words Model
* RNN Models using Word Embeddings

In [236]:
%load_ext autoreload
%autoreload 2

import torch
import torch.utils.data as tud
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchtext
from torchtext import data

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('stopwords')

from collections import Counter, defaultdict
import operator
import os, math
import random
import copy
import string
import multiprocessing as mp

from split_data import split_data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package punkt to /home/antimony/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/antimony/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# set the random seeds so the experiments can be replicated exactly
random.seed(72689)
np.random.seed(72689)
torch.manual_seed(72689)
if torch.cuda.is_available():
    torch.cuda.manual_seed(72689)

# Global class labels.
POS_LABEL = 'up'
NEG_LABEL = 'down'

**Reading in all the Data**

In [3]:
all_data = pd.read_csv("ProcessedData/CombinedData.csv")
all_data.drop(columns=['Unnamed: 0'], inplace=True)
all_data.head()

,Title,Date,Content,OpenMove,CloseMove
0,Top U.S. General Praises Iran-Backed Shiite Mi...,2017-01-04,The top commander of the U.S.-led coalition ag...,1.0,1.0
1,Extremists Turn to a Leader to Protect Western...,2017-01-04,As the founder of the Traditionalist Worker Pa...,1.0,1.0
2,How Julian Assange evolved from pariah to paragon,2017-01-04,President-elect Donald Trump tweeted some pra...,1.0,1.0
3,House panel recommends cutting funding for Pla...,2017-01-04,A House panel formed by Republicans to invest...,1.0,1.0
4,Missouri Bill: Gun-Banning Businesses Liable f...,2017-01-04,As Missouri lawmakers convene for the 2017 leg...,1.0,1.0


*Using a Small Subset of Data fro Development*

In [216]:
data_sample = all_data.sample(1000, random_state=68)
data_sample.reset_index(drop=True, inplace=True)
data_sample.head()

,Title,Date,Content,OpenMove,CloseMove
0,Another government shutdown over Obamacare? On...,2017-04-24,"\nThis is the web version of VoxCare, a daily ...",1.0,1.0
1,Tourists Helped Fatten Him Up; Now Thai Monkey...,2017-05-19,"[Whether he likes it or not, a morbidly obese ...",1.0,1.0
2,"Indian Premier, in Israel Visit, Seeks to Brea...",2017-07-05,JERUSALEM — Prime Minister Benjamin Netanyahu ...,0.0,0.0
3,Kiribati Ends Aerial Search for Missing Ferry ...,2018-03-21,"WELLINGTON, New Zealand — The aerial search fo...",0.0,0.0
4,The American Model,2017-07-28,Amidst a string of pat introductory refl...,0.0,0.0


### Preprocessing the Data For Feeding Into The Model

Preprocessing Involves (in our case):
* Turning All Words into lower/upper case, Normalization
* removing punctuations, accent marks and other diacritics
* removing stop words, sparse terms, and particular words
* Stemming using a Porter Stemmer from NLTK 

In [217]:
# Removing all Punctuation
def remove_punctuation(text):
    more_puncs = '—'+ '’'+ '“'+ '”'+ '…'
    return text.translate(str.maketrans('', '', string.punctuation+more_puncs))

# Removing all Stop Words
def remove_stopwords(text, stop_words):
    text = word_tokenize(text)
    return  " ".join([i for i in text if i not in stop_words])

def stem(text, stemmer):
    text = word_tokenize(text)
    return " ".join([stemmer.stem(i) for i in text])

The pre_process function below performs all the preprocessing we defined above. 

In [218]:
def pre_process(df):
    # Normalization
    df['Title'] = df['Title'].str.lower()
    df['Content'] = df['Content'].str.lower()

    # Removing Punctuation
    df['Title'] = df['Title'].apply(remove_punctuation)
    df['Content'] = df['Content'].apply(remove_punctuation)
    
    STOP_WORDS = set(stopwords.words('english'))
    # Remove Stopwords
    df['Title'] = df['Title'].apply(remove_stopwords, args=(STOP_WORDS, ))
    df['Content'] = df['Content'].apply(remove_stopwords, args=(STOP_WORDS, ))

    # Stemming
    stemmer = PorterStemmer()
    df['Title'] = df['Title'].apply(stem, args=(stemmer,))
    df['Content'] = df['Content'].apply(stem, args=(stemmer,))

    return df

**We run the pre_process function in parallel to make it faster using the Multi-Processing Module**

In [219]:
# Processing in Parallel
n_threads = mp.cpu_count()-1
data_pieces = np.array_split(data_sample, n_threads)

pool = mp.Pool(n_threads)
data_sample = pd.concat(pool.map(pre_process, data_pieces))
pool.close()
pool.join()

data_sample.head()

,Title,Date,Content,OpenMove,CloseMove
0,anoth govern shutdown obamacar trump want,2017-04-24,web version voxcar daili newslett vox latest t...,1.0,1.0
1,tourist help fatten thai monkey diet,2017-05-19,whether like morbidli obes wild monkey thailan...,1.0,1.0
2,indian premier israel visit seek break barrier...,2017-07-05,jerusalem prime minist benjamin netanyahu long...,0.0,0.0
3,kiribati end aerial search miss ferri passeng ...,2018-03-21,wellington new zealand aerial search ferri kir...,0.0,0.0
4,american model,2017-07-28,amidst string pat introductori reflect recent ...,0.0,0.0


### Preparing Data for the CBOW Model

* Building the Vocabulary (Using Spacy) | **MAX_VOCAB_SIZE** = 25000
* Splitting the data for Test and Training


In [120]:
SEED = 68

In [208]:
TEXT = data.Field(tokenize = 'spacy')
LABEL = data.LabelField(dtype = torch.float)

Splitting the Data and Storing it such that torch text can easily ingest it.

In [237]:
split_data(df=data_sample[['Content', 'CloseMove']],prefix='dev',seed=SEED)

In [238]:
train_data, val, test = data.TabularDataset.splits(
        path='./ProcessedData/', train='dev_train.csv',
        validation='dev_val.csv', test='dev_test.csv', format='csv',
        fields=[('Text', TEXT), ('Label', LABEL)])


In [239]:
MAX_VOCAB_SIZE = 25000
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

`Sanity Check for Vocabulary and Labels`

In [244]:
(TEXT.vocab.itos[:10]), (LABEL.vocab.itos)

(['<unk>',
  '<pad>',
  'said',
  'trump',
  'mr',
  'nt',
  'would',
  'one',
  'year',
  'peopl'],
 ['1.0', '0.0'])